In [ ]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter, argrelextrema
import io
import base64

app = dash.Dash(__name__)

# ===== 功能函数 =====
def preprocess_and_extract_features(df, filename):
    df_charge = df[df['Condition'] == 1].copy()
    df_discharge = df[df['Condition'] == 2].copy()

    result = {
        "filename": filename,
        "mean_voltage_charge": np.nan,
        "mean_voltage_discharge": np.nan,
        "ir_drop": np.nan,
        "plateau_count": np.nan,
        "plateau_positions": [],
        "plateau_widths": []
    }

    def interpolate_and_extract(sub_df, soc_col):
        sub_df = sub_df[[soc_col, 'Voltage[V]', 'Capacity[mAh]']].dropna()
        sub_df = sub_df.sort_values(by=soc_col)
        x = sub_df[soc_col].values
        v = sub_df['Voltage[V]'].values
        q = sub_df['Capacity[mAh]'].values

        grid = np.arange(0, 101, 1)
        if len(np.unique(x)) < 10:
            return None, None, None

        v_interp = np.interp(grid, x, v)
        q_interp = np.interp(grid, x, q)
        return grid, v_interp, q_interp

    soc_grid, v_c, q_c = interpolate_and_extract(df_charge, 'SoC[%]')
    if soc_grid is not None:
        dq = np.gradient(q_c)
        result['mean_voltage_charge'] = np.sum(v_c * dq) / np.sum(dq)

        v_smooth = savgol_filter(v_c, 11, 2)
        peaks = argrelextrema(v_smooth, np.less)[0]
        result['plateau_count'] = len(peaks)
        result['plateau_positions'] = soc_grid[peaks].tolist()
        widths = np.diff(soc_grid[peaks]) if len(peaks) > 1 else []
        result['plateau_widths'] = list(widths)

    dod_grid, v_d, q_d = interpolate_and_extract(df_discharge, 'DoD[%]')
    if dod_grid is not None:
        dq = np.gradient(q_d)
        result['mean_voltage_discharge'] = np.sum(v_d * dq) / np.sum(dq)
        early_v = v_d[dod_grid <= 5]
        if len(early_v) > 1:
            result['ir_drop'] = np.max(early_v) - np.min(early_v)

    return soc_grid, q_c, dod_grid, q_d, result

# ====== 页面布局 ======
app.layout = html.Div([
    html.H3("🔋 SoC/DoD vs Capacity 分析工具"),
    dcc.Upload(
        id='upload-data',
        children=html.Div(['拖拽或点击上传 CSV 文件（可多选）']),
        style={
            'width': '98%', 'height': '60px', 'lineHeight': '60px',
            'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
            'textAlign': 'center', 'margin': '10px'
        },
        multiple=True
    ),
    dcc.Graph(id='soc-graph'),
    dcc.Graph(id='dod-graph'),
    dash_table.DataTable(id='feature-table', style_table={'overflowX': 'auto'},
                         style_cell={'textAlign': 'center', 'minWidth': '100px'}),
    html.Button("下载特征为 CSV", id="download-btn", n_clicks=0),
    dcc.Download(id="download-data")
])

# ====== Callback ======
@app.callback(
    Output('soc-graph', 'figure'),
    Output('dod-graph', 'figure'),
    Output('feature-table', 'data'),
    Output('feature-table', 'columns'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_output(contents, filenames):
    if contents is None:
        return {}, {}, [], []

    soc_fig = {
        "data": [],
        "layout": {"title": "充电曲线：SoC vs Capacity", "xaxis": {"title": "SoC[%]"}, "yaxis": {"title": "Capacity [mAh]"}}
    }
    dod_fig = {
        "data": [],
        "layout": {"title": "放电曲线：DoD vs Capacity", "xaxis": {"title": "DoD[%]"}, "yaxis": {"title": "Capacity [mAh]"}}
    }
    all_features = []

    for c, name in zip(contents, filenames):
        content_type, content_string = c.split(',')
        decoded = base64.b64decode(content_string)
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

        soc_grid, q_c, dod_grid, q_d, features = preprocess_and_extract_features(df, name)
        all_features.append(features)

        if soc_grid is not None:
            soc_fig['data'].append({"x": soc_grid, "y": q_c, "mode": "lines", "name": name})
        if dod_grid is not None:
            dod_fig['data'].append({"x": dod_grid, "y": q_d, "mode": "lines", "name": name})

    df_feat = pd.DataFrame(all_features)
    columns = [{"name": col, "id": col} for col in df_feat.columns]
    return soc_fig, dod_fig, df_feat.to_dict("records"), columns

# ===== 下载按钮回调 =====
@app.callback(
    Output("download-data", "data"),
    Input("download-btn", "n_clicks"),
    State('feature-table', 'data'),
    prevent_initial_call=True
)
def download_features(n_clicks, table_data):
    df = pd.DataFrame(table_data)
    return dcc.send_data_frame(df.to_csv, filename="特征提取结果.csv", index=False)

if __name__ == '__main__':
    app.run_server(debug=True)
